# 1. Install Dependencies

In [ ]:
!pip install -q -U "dotenv==0.9.9" "langchain-google-genai==2.1.8" "crewai==0.150.0" "google-adk==1.8.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 69.6 MB/s eta 0

# 2. Import Packages

In [ ]:
import os, getpass
import asyncio
import nest_asyncio
from typing import List
from dotenv import load_dotenv
import logging


from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool as langchain_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor

from crewai import Agent as CrewAgent, Task, Crew
from crewai.tools import tool as crew_tool

from google.adk.agents import Agent as ADKAgent, LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.adk.code_executors import BuiltInCodeExecutor
from google.genai import types

In [ ]:
# Basic logging setup helps in debugging and tracking to execution.
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


# 2. Setup API Keys

In [ ]:
# UNCOMMENT
# Prompt the user securely and set API keys as an environment variables
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key: ")
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your Google API key: ··········
Enter your OpenAI API key: ··········


# 3. LangChain

## 3.1 Setup LLM

In [ ]:
try:
   # A model with function/tool calling capabilities is required.
   llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
   print(f"✅ Language model initialized: {llm.model}")
except Exception as e:
   print(f"🛑 Error initializing language model: {e}")
   llm = None

✅ Language model initialized: models/gemini-2.0-flash


## 3.2 Define Search Tool

In [ ]:
# --- Define a Tool ---
@langchain_tool
def search_information(query: str) -> str:
   """
   Provides factual information on a given topic. Use this tool to find answers to phrases
   like 'capital of France' or 'weather in London?'.
   """
   print(f"\n--- 🛠️ Tool Called: search_information with query: '{query}' ---")
   # Simulate a search tool with a dictionary of predefined results.
   simulated_results = {
       "weather in london": "The weather in London is currently cloudy with a temperature of 15°C.",
       "capital of france": "The capital of France is Paris.",
       "population of earth": "The estimated population of Earth is around 8 billion people.",
       "tallest mountain": "Mount Everest is the tallest mountain above sea level.",
       "default": f"Simulated search result for '{query}': No specific information found, but the topic seems interesting."
   }
   result = simulated_results.get(query.lower(), simulated_results["default"])
   print(f"--- TOOL RESULT: {result} ---")
   return result


tools = [search_information]

## 3.3 Tool Calling Agent

In [ ]:
# --- Create a Tool-Calling Agent ---
if llm:
   # This prompt template requires an `agent_scratchpad` placeholder for the agent's internal steps.
   agent_prompt = ChatPromptTemplate.from_messages([
       ("system", "You are a helpful assistant."),
       ("human", "{input}"),
       ("placeholder", "{agent_scratchpad}"),
   ])


   # Create the agent, binding the LLM, tools, and prompt together.
   agent = create_tool_calling_agent(llm, tools, agent_prompt)


   # AgentExecutor is the runtime that invokes the agent and executes the chosen tools.
   # The 'tools' argument is not needed here as they are already bound to the agent.
   agent_executor = AgentExecutor(agent=agent, verbose=True, tools=tools)

## 3.4 Run Agent

In [ ]:
async def run_agent_with_tool(query: str):
   """Invokes the agent executor with a query and prints the final response."""
   print(f"\n--- 🏃 Running Agent with Query: '{query}' ---")
   try:
       response = await agent_executor.ainvoke({"input": query})
       print("\n--- ✅ Final Agent Response ---")
       print(response["output"])
   except Exception as e:
       print(f"\n🛑 An error occurred during agent execution: {e}")


async def main():
   """Runs all agent queries concurrently."""
   tasks = [
       run_agent_with_tool("What is the capital of France?"),
       run_agent_with_tool("What's the weather like in London?"),
       run_agent_with_tool("Tell me something about dogs.") # Should trigger the default tool response
   ]
   await asyncio.gather(*tasks)



nest_asyncio.apply()
asyncio.run(main())


--- 🏃 Running Agent with Query: 'What is the capital of France?' ---

--- 🏃 Running Agent with Query: 'What's the weather like in London?' ---

--- 🏃 Running Agent with Query: 'Tell me something about dogs.' ---


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...

Invoking: `search_information` with `{'query': 'weather in London'}`



--- 🛠️ Tool Called: search_information with query: 'weather in London' ---
--- TOOL RESULT: The weather in London is currently cloudy with a temperature of 15°C. ---
The weather in London is currently cloudy with a temperature of 15°C.
Invoking: `search_information` with `{'query': 'characteristics of dogs'}`



--- 🛠️ Tool Called: search_information with query: 'characteristics of dogs' ---
--- TOOL RESULT: Simulated search result for 'characteristics of dogs': No specific information found, but the topic seems interesting. ---
Simulated search result for 'characteristics of dogs': No s

# 4. Crew AI

## 4.1 Define Stock Price Search Tool

In [ ]:
# --- 1. Refactored Tool: Returns Clean Data ---
# The tool now returns raw data (a float) or raises a standard Python error.
# This makes it more reusable and forces the agent to handle outcomes properly.
@crew_tool("Stock Price Lookup Tool")
def get_stock_price(ticker: str) -> float:
   """
   Fetches the latest simulated stock price for a given stock ticker symbol.
   Returns the price as a float. Raises a ValueError if the ticker is not found.
   """
   logging.info(f"Tool Call: get_stock_price for ticker '{ticker}'")
   simulated_prices = {
       "AAPL": 178.15,
       "GOOGL": 1750.30,
       "MSFT": 425.50,
   }
   price = simulated_prices.get(ticker.upper())


   if price is not None:
       return price
   else:
       # Raising a specific error is better than returning a string.
       # The agent is equipped to handle exceptions and can decide on the next action.
       raise ValueError(f"Simulated price for ticker '{ticker.upper()}' not found.")






## 4.2 Setup Crew Entities

In [ ]:
# --- 2. Define the Agent ---
# The agent definition remains the same, but it will now leverage the improved tool.
financial_analyst_agent = CrewAgent(
 role='Senior Financial Analyst',
 goal='Analyze stock data using provided tools and report key prices.',
 backstory="You are an experienced financial analyst adept at using data sources to find stock information. You provide clear, direct answers.",
 verbose=True,
 tools=[get_stock_price],
 # Allowing delegation can be useful, but is not necessary for this simple task.
 allow_delegation=False,
)


# --- 3. Refined Task: Clearer Instructions and Error Handling ---
# The task description is more specific and guides the agent on how to react
# to both successful data retrieval and potential errors.
analyze_aapl_task = Task(
 description=(
     "What is the current simulated stock price for Apple (ticker: AAPL)? "
     "Use the 'Stock Price Lookup Tool' to find it. "
     "If the ticker is not found, you must report that you were unable to retrieve the price."
 ),
 expected_output=(
     "A single, clear sentence stating the simulated stock price for AAPL. "
     "For example: 'The simulated stock price for AAPL is $178.15.' "
     "If the price cannot be found, state that clearly."
 ),
 agent=financial_analyst_agent,
)


# --- 4. Formulate the Crew ---
# The crew orchestrates how the agent and task work together.
financial_crew = Crew(
 agents=[financial_analyst_agent],
 tasks=[analyze_aapl_task],
 verbose=True # Set to False for less detailed logs in production
)


## 4.3 Run Crew Agent

In [ ]:


# --- 5. Run the Crew within a Main Execution Block ---
# Using a __name__ == "__main__": block is a standard Python best practice.
def main():
   """Main function to run the crew."""
   # Check for API key before starting to avoid runtime errors.
   if not os.environ.get("OPENAI_API_KEY"):
       print("ERROR: The OPENAI_API_KEY environment variable is not set.")
       print("Please set it before running the script.")
       return


   print("\n## Starting the Financial Crew...")
   print("---------------------------------")

   # The kickoff method starts the execution.
   result = financial_crew.kickoff()


   print("\n---------------------------------")
   print("## Crew execution finished.")
   print("\nFinal Result:\n", result)


if __name__ == "__main__":
   main()



## Starting the Financial Crew...
---------------------------------


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 22e3da43-7199-40fd-8817-9aa4bc877eac                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Financial Analyst                                                                                │
│                                                                                                                 │
│  Task: What is the current simulated stock price for Apple (ticker: AAPL)? Use the 'Stock Price Lookup Tool'    │
│  to find it. If the ticker is not found, you must report that you were unable to retrieve the price.            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Financial Analyst                                                                                │
│                                                                                                                 │
│  Thought: Thought: I need to find the current simulated stock price for Apple (ticker: AAPL).                   │
│                                                                                                                 │
│  Using Tool: Stock Price Lookup Tool                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Financial Analyst                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The simulated stock price for AAPL is $178.15.                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 3ae83180-2ac2-456f-aff6-2ff30a3e9f89                                                                     │
│  Agent: Senior Financial Analyst                                                                                │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 22e3da43-7199-40fd-8817-9aa4bc877eac                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The simulated stock price for AAPL is $178.15.                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


---------------------------------
## Crew execution finished.

Final Result:
 The simulated stock price for AAPL is $178.15.


# 5. Google Agent Development Kit (ADK)

## 5.1 Agent with Search Tool

In [ ]:
# Define variables required for Session setup and Agent execution
APP_NAME="Google Search_agent"
USER_ID="user1234"
SESSION_ID="1234"



# Define Agent with access to search tool
root_agent = ADKAgent(
   name="basic_search_agent",
   model="gemini-2.0-flash-exp",
   description="Agent to answer questions using Google Search.",
   instruction="I can answer your questions by searching the internet. Just ask me anything!",
   tools=[google_search] # Google Search is a pre-built tool to perform Google searches.
)




# Agent Interaction
async def call_agent(query):
   """
   Helper function to call the agent with a query.
   """

   # Session and Runner
   session_service = InMemorySessionService()
   session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
   runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)

   content = types.Content(role='user', parts=[types.Part(text=query)])
   events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)


   for event in events:
       if event.is_final_response():
           final_response = event.content.parts[0].text
           print("Agent Response: ", final_response)

nest_asyncio.apply()

asyncio.run(call_agent("what's the latest ai news?"))

Agent Response:  Here's a summary of recent AI news:

*   **AI and Job Markets**: Sam Altman (OpenAI CEO) has warned of potential job losses and national security threats due to AI. Research indicates AI is changing the job landscape, prompting nations to prepare their workforces.
*   **AI Models and Advancements**: Google made AI models faster and cheaper. Google's Gemini 2.5 model aims for 'intelligence per dollar'. Google also introduced AlphaGenome for understanding the human genome and launched Weather Lab to support better tropical cyclone prediction with AI.
*   **AI in Different Sectors**: AI is rewriting the rules of the insurance industry. AI is helping students learn and scientists understand the human body better.
*   **AI Ethics and Safety**: There are growing concerns about AI safety. A study revealed that even the most powerful AI models can make basic errors in navigating ethical medical decisions.
*   **Talent Acquisition**: Meta is actively hiring AI researchers, incl

## 5.2 Agent with Code Tool

In [ ]:
# Define variables required for Session setup and Agent execution
APP_NAME="calculator"
USER_ID="user1234"
SESSION_ID="session_code_exec_async"


# Agent Definition
code_agent = LlmAgent(
   name="calculator_agent",
   model="gemini-2.0-flash",
   code_executor=BuiltInCodeExecutor(),
   instruction="""You are a calculator agent.
   When given a mathematical expression, write and execute Python code to calculate the result.
   Return only the final numerical result as plain text, without markdown or code blocks.
   """,
   description="Executes Python code to perform calculations.",
)

# Agent Interaction (Async)
async def call_agent_async(query):

   # Session and Runner
   session_service = InMemorySessionService()
   session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
   runner = Runner(agent=code_agent, app_name=APP_NAME, session_service=session_service)

   content = types.Content(role='user', parts=[types.Part(text=query)])
   print(f"\n--- Running Query: {query} ---")
   final_response_text = "No final text response captured."
   try:
       # Use run_async
       async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
           print(f"Event ID: {event.id}, Author: {event.author}")

           # --- Check for specific parts FIRST ---
           # has_specific_part = False
           if event.content and event.content.parts and event.is_final_response():
               for part in event.content.parts: # Iterate through all parts
                   if part.executable_code:
                       # Access the actual code string via .code
                       print(f"  Debug: Agent generated code:\n```python\n{part.executable_code.code}\n```")
                       has_specific_part = True
                   elif part.code_execution_result:
                       # Access outcome and output correctly
                       print(f"  Debug: Code Execution Result: {part.code_execution_result.outcome} - Output:\n{part.code_execution_result.output}")
                       has_specific_part = True
                   # Also print any text parts found in any event for debugging
                   elif part.text and not part.text.isspace():
                       print(f"  Text: '{part.text.strip()}'")
                       # Do not set has_specific_part=True here, as we want the final response logic below

               # --- Check for final response AFTER specific parts ---
               text_parts = [part.text for part in event.content.parts if part.text]
               final_result = "".join(text_parts)
               print(f"==> Final Agent Response: {final_result}")

   except Exception as e:
       print(f"ERROR during agent run: {e}")
   print("-" * 30)

# Main async function to run the examples
async def main():
   await call_agent_async("Calculate the value of (5 + 7) * 3")
   await call_agent_async("What is 10 factorial?")


# Execute the main async function
try:
   nest_asyncio.apply()
   asyncio.run(main())
except RuntimeError as e:
   # Handle specific error when running asyncio.run in an already running loop (like Jupyter/Colab)
   if "cannot be called from a running event loop" in str(e):
       print("\nRunning in an existing event loop (like Colab/Jupyter).")
       print("Please run `await main()` in a notebook cell instead.")
       # If in an interactive environment like a notebook, you might need to run:
       # await main()
   else:
       raise e # Re-raise other runtime errors


--- Running Query: Calculate the value of (5 + 7) * 3 ---


Event ID: 23696e40-5e9d-4da3-89f5-032971769c2b, Author: calculator_agent
  Debug: Agent generated code:
```python
print((5 + 7) * 3)

```
  Debug: Code Execution Result: Outcome.OUTCOME_OK - Output:
36

  Text: '36'
==> Final Agent Response: 36

------------------------------

--- Running Query: What is 10 factorial? ---


Event ID: 11d25af6-8864-497f-8846-7c054503a100, Author: calculator_agent
  Debug: Agent generated code:
```python
import math
print(math.factorial(10))

```
  Debug: Code Execution Result: Outcome.OUTCOME_OK - Output:
3628800

  Text: '3628800'
==> Final Agent Response: 3628800

------------------------------


End of notebook!